In [1]:
import tensorflow as tf
import numpy as np

In [2]:
graph = tf.Graph()

In [3]:
with graph.as_default():
    with tf.name_scope("Variables"):
        global_step =tf.Variable(0,dtype=tf.int32,trainable=False,name="global_step")
        total_output=tf.Variable(0.0,dtype=tf.float32,trainable=False,name="total_output")
        
    with tf.name_scope("transformation"):
        with tf.name_scope("input"):
            a=tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")
            
        with tf.name_scope("intermediate_layer"):
            b=tf.reduce_prod(a,name="product_b")
            c=tf.reduce_sum(a,name="sum_c")
        
        with tf.name_scope("output"):
            output=tf.add(b,c,name="output")
    with tf.name_scope("update"):
        #Incrementsthetotal_output Variable bythe latestoutput
        update_total=total_output.assign_add(output)
        # Increments this variable for every run of graph
        increment_step = global_step.assign_add(1)
    with tf.name_scope("summaries"):
        avg=tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
        #Creates summaries for output node
        tf.summary.scalar('Output',output)
        tf.summary.scalar("total_summary",update_total)
        tf.summary.scalar('Average_Summary',avg)
    with tf.name_scope('ops'):
        init = tf.global_variables_initializer()
        merged_summaries = tf.summary.merge_all()







In [4]:
sess = tf.Session(graph=graph)
writer=tf.summary.FileWriter('./graphv1',	graph=graph)

In [5]:
sess.run(init)

In [6]:
def run_graph(input_tensor):
    input_tensor = np.asarray(input_tensor)
    #print input_tensor
    feed_dict ={a:input_tensor}
    #print feed_dict
    _,step,summary=sess.run([output,increment_step,merged_summaries],feed_dict=feed_dict)
    writer.add_summary(summary,global_step=step)


In [7]:
run_graph([2.0,3.0])
run_graph([3,4])
run_graph([2,2])
run_graph([2,3])

In [8]:
writer.flush()
writer.close()
sess.close()